In [ ]:
!pip install timm tqdm

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import timm
from tqdm import tqdm


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set data directory
data_dir = "/content/drive/MyDrive/ISM/data"

In [ ]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = ImageFolder(root=data_dir, transform=transform)

# Split dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

In [ ]:
# Create DataLoader
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=2)

# Load pre-trained ResNet50 from timm
model = timm.create_model("resnet50", pretrained=True) # try with 18, 34 and 50
model.fc = nn.Linear(model.fc.in_features, 4)

# Move model to device
model = model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0, 1.0, 3.0]).to(device))

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Testing loop
model.eval()
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

In [ ]:
torch.save(model.state_dict(), 'resnet_model.pth')